In [2]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

# Override/set credentials in env var
os.environ['CWD'] = str(Path(os.getcwd()).parent)

# Base paths
cwd = Path(os.environ['CWD'])
dir_data = cwd / 'data'


import numpy as np
from PIL import Image

In [4]:
# !pip install "ray[serve]==1.11.0"

In [3]:
import ray
from ray import serve
ray.init(dashboard_host="0.0.0.0")
serve.start()

2022-04-05 12:17:59,537	INFO services.py:1414 -- View the Ray dashboard at http://172.18.0.2:8265
2022-04-05 12:17:59,540	WARNING services.py:1919 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=1.71gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
(ServeController pid=45678) 2022-04-05 12:18:01,265	INFO checkpoint_path.py:16 -- Using RayInternalKVStore for controller checkpoint and recovery.
(ServeController pid=45678) 2022-04-05 12:18:01,373	INFO http_state.py:101 -- Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:fULZbQ:SERVE_PROXY_ACTOR-node:172.18.0.2-0' on node 'node:172.18.0.2-0' listening on '127.0.0.1:8000'
2022-04-05 12:18:01,632	INFO ap

(HTTPProxyActor pid=45681) INFO:     Started server process [45681]


In [4]:
from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier

# Train model.
iris_dataset = load_iris()
model = GradientBoostingClassifier()
model.fit(iris_dataset["data"], iris_dataset["target"])

GradientBoostingClassifier()

In [15]:
from time import sleep

# @serve.deployment(route_prefix="/iris")
class BoostingModel:
    def __init__(self, model):
        self.model = model
        self.label_list = iris_dataset["target_names"].tolist()
        self.n_calls = 0

    async def __call__(self, request):
        if type(request) == dict: # For local pred
            payload = request["vector"]
        else: # For HTTP pred
            payload = (await request.json())["vector"]
        print(f"Received http request with data {payload}")
        sleep(6)
        prediction = self.model.predict([payload])[0]
        human_name = self.label_list[prediction]
        self.n_calls += 1
        return {
            "result": human_name,
            "n_calls": self.n_calls
        }

In [6]:
# Try actor locally
# Instantiate object
boosting_model = BoostingModel(model)

In [7]:
# Make pred
d_ret = await boosting_model({"vector": [1.2, 1.0, 1.1, 0.9]})

Received http request with data [1.2, 1.0, 1.1, 0.9]


In [8]:
d_ret

{'result': 'versicolor'}

In [16]:
# Deploy the same actor
serve.deployment(route_prefix="/iris")(BoostingModel).deploy(model)

2022-04-05 12:26:10,299	INFO api.py:262 -- Updating deployment 'BoostingModel'. component=serve deployment=BoostingModel
(ServeController pid=45678) 2022-04-05 12:26:10,313	INFO deployment_state.py:882 -- Stopping 1 replicas of deployment 'BoostingModel' with outdated versions. component=serve deployment=BoostingModel
(ServeController pid=45678) 2022-04-05 12:26:12,525	INFO deployment_state.py:920 -- Adding 1 replicas to deployment 'BoostingModel'. component=serve deployment=BoostingModel
2022-04-05 12:26:13,818	INFO api.py:275 -- Deployment 'BoostingModel' is ready at `http://127.0.0.1:8000/iris`. component=serve deployment=BoostingModel


In [47]:
# Query it!
import requests
sample_request_input = {"vector": [1.2, 1.0, 0, 0.9]}
response = requests.get(
    "http://localhost:8000/iris", json=sample_request_input)
print(response.text)

(BoostingModel pid=49188) Received http request with data [1.2, 1.0, 0, 0.9]
(BoostingModel pid=49223) Received http request with data [1.2, 1.0, 1.1, 0.9]
{
  "result": "versicolor",
  "n_calls": 4
}
(BoostingModel pid=49188) Received http request with data [1.2, 1.0, 1.1, 0.9]


In [ ]:
# Query it!
import requests
sample_request_input = {"vector": [1.2, 1.0, 1.1, 0.9]}
response = requests.get(
    "http://localhost:8000/iris", json=sample_request_input)
print(response.text)

In [30]:
serve.list_deployments()

{'BoostingModel': Deployment(name=BoostingModel,version=None,route_prefix=/iris)}

In [40]:
serve.get_deployment('BoostingModel').options(num_replicas=2).deploy()

2022-04-05 13:06:30,311	INFO api.py:262 -- Updating deployment 'BoostingModel'. component=serve deployment=BoostingModel
(ServeController pid=45678) 2022-04-05 13:06:30,331	INFO deployment_state.py:920 -- Adding 1 replicas to deployment 'BoostingModel'. component=serve deployment=BoostingModel
(ServeController pid=45678) 2022-04-05 13:06:31,616	INFO deployment_state.py:882 -- Stopping 1 replicas of deployment 'BoostingModel' with outdated versions. component=serve deployment=BoostingModel
(ServeController pid=45678) 2022-04-05 13:06:33,742	INFO deployment_state.py:920 -- Adding 1 replicas to deployment 'BoostingModel'. component=serve deployment=BoostingModel
2022-04-05 13:06:34,911	INFO api.py:275 -- Deployment 'BoostingModel' is ready at `http://127.0.0.1:8000/iris`. component=serve deployment=BoostingModel
